In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import h5py

In [2]:
#vector norm
def L2Norm(S):
    return torch.sqrt(S[0]**2+S[1]**2+S[2]**2)

# normalizing a vector to a unit vector
def Vnorm(V):
    v_i = V[0]
    v_j = V[1]
    v_k = V[2]
    
    norm = L2Norm(V)
    return torch.Tensor([[V[0]/norm, V[1]/norm, V[2]/norm]]).t()

In [3]:
#RK4 implementation
def RK4(function, pos_vec, vel_vec, timestep):
    f1_pos, f1_vel = function(pos_vec, vel_vec)
    f2_pos, f2_vel = function(pos_vec+timestep*f1_pos/2, vel_vec+timestep*f1_vel/2)
    f3_pos, f3_vel = function(pos_vec+timestep*f2_pos/2, vel_vec+timestep*f2_vel/2)
    f4_pos, f4_vel = function(pos_vec+timestep*f3_pos,vel_vec+timestep*f3_vel)
    new_pos = pos_vec + timestep/6 * (f1_pos + 2*f2_pos + 2*f3_pos+f4_pos)
    new_vel = vel_vec + timestep/6 * (f1_vel + 2*f2_vel + 2*f3_vel+f4_vel)
    return new_pos, new_vel

In [4]:
# Equations of motion
def propogate(position_vector, velocity_vector):
    GM = 3.986004418e14
    g = GM/L2Norm(position_vector)**2
    s_hat = Vnorm(position_vector)
    d_vel = torch.mul(-s_hat,g)
    
    return velocity_vector, d_vel

In [5]:
def plot_orbit_2d(S_out, collision):
    plt.plot(S_out[:,0],S_out[:,1])
    ax = plt.gca().set_aspect('equal')
    
    #plot last point before surface impact
    if collision:
        plt.plot(S_out[-1,0],S_out[-1,1],marker='o',markeredgecolor='red')
        
    #plot earth
    r = 6.3781e6
    theta = np.arange(0,2*np.pi,0.01)
    x_earth = r*np.cos(theta)
    y_earth = r*np.sin(theta)
    plt.plot(x_earth,y_earth)
    
    #label the plot
    plt.legend(['Orbit','Earth'])
    
    #Save the image
    time = datetime.now()
    t_format = "%H%M%S"
    plt.savefig(f"{time.strftime(t_format)}.png")

In [7]:
def save_sim_data(radius_earth, altitude, initial_vel, time_step, total_time):
    # generate timestamp
    time = datetime.now()
    t_format = "%m%d%Y-%H%M%S"
    datetime_prefix = f"{time.strftime(t_format)}"
    print(datetime_prefix)
    
    # Open HDF file
    f = h5py.File(f"{datetime_prefix}.hdf5", 'w')
    print(list(f.keys()))
    
    #save configuration data
    dset = f.create_dataset("config", (5,))

In [8]:
def orbital_simulation(radius_earth, altitude, initial_vel, time_step, total_time):
    S = torch.Tensor([[radius_earth+altitude,0,0]]).t() # initial orbital altitude
    V = torch.Tensor([[initial_vel/2,initial_vel,0]]).t() # initial orbital velocity
    S_out = torch.Tensor()
    collision = False

    for t in range(total_time):
        S, V = RK4(propogate,S,V,time_step)
        S_out = torch.cat((S_out,S.t()))

        #check for collision
        norm = L2Norm(S)
        if norm < radius_earth:
            collision = True;
            break;
    
    save_sim_data(radius_earth, altitude, initial_vel, time_step, total_time)

In [9]:
radius_earth = 6.3781e6 #radius in m
altitude = 200e3 # altitude in m
initial_vel = 8.8e3 # velocity in m/s
time_step = 60 # seconds
total_time = 100 # minutes

orbital_simulation(radius_earth, altitude, initial_vel, time_step, total_time)

04302023-184341
[]
